# API Queries

This notebook was created to help document issues found with the Matrix API.

We have observed unusual behaviour with the API depending on whether we provide a **status** parameter in the query.

## Define standard functions

In [ ]:
import pandas as pd
import requests
import re
import json
from arrow_pd_parser import writer, reader
import python_scripts.s3_utils as s3_utils
from datetime import datetime, timedelta
from mojap_metadata import Metadata



from logging import getLogger

logger = getLogger(__name__)


def read_json(file_path: str) -> dict:
    """Reads a json file in as a dictionary

    Parameters
    ----------
    file_path :
        file path of the JSON to read from

    Returns
    -------
        dictionary representing the json file
    """
    f = open(file_path)
    return json.loads(f.read())


def get_secrets() -> dict:
    return s3_utils.read_json_from_s3("alpha-dag-matrix/api_secrets/secrets.json")


def matrix_authenticate(session: requests.Session) -> requests.Session:
    # Add this before the error occurs
    for handler in logger.handlers:
        for record in handler.records:
            print(record.__dict__)
    secrets = get_secrets()
    username = secrets["username"]
    password = secrets["password"]

    url = "https://app.matrixbooking.com/api/v1/user/login"
    session.post(url, json={"username": username, "password": password})
    return session


def get_booking_categories(session: requests.Session) -> pd.DataFrame:
    """Returns pandas dataframe containing all booking categories
    that are available to organisation

     Parameters:
    session (requests.sessions.Session): Authenticated session to
        matrix booking API
    """

    # Booking categories API url
    url_booking_cats = "https://app.matrixbooking.com/api/v1/category"

    # Make request and create dataframe
    res = requests.get(url_booking_cats, cookies=session.cookies).json()
    df_booking_categories = pd.json_normalize(res)

    return df_booking_categories


def make_booking_params(
    time_from: str,
    time_to: str,
    booking_categories: str,
    status: str = None,
    pageSize: int = None,
    pageNum: int = 0,
) -> dict:
    params = {
        "f": time_from,
        "t": time_to,
        "bc": booking_categories,
        "status": status,
        "include": ["audit", "locations"],
        "pageSize": pageSize,
        "pageNum": pageNum,
    }
    return params


def get_payload(session, url, parameters):
    resp = session.get(url=url, cookies=session.cookies, params=parameters)
    logger.debug(f"GET {resp.url}")
    logger.debug(f"response status code: {resp.status_code}")
    return resp


def split_s3_path(s3_path: str) -> tuple[str]:
    """Splits an s3 file path into a bucket and key

    Parameters
    ----------
    s3_path :
        The full (incl s3://) path of a file.

    Returns
    -------
        Tuple of the bucket name and key (file path) within that bucket.
    """
    if s3_path[:2] != "s3":
        raise ValueError("S3 file path should start with 's3://'.")
    path_split = s3_path.split("/")
    bucket = path_split[2]
    key = "/".join(path_split[3:])
    return bucket, key


def get_scrape_dates(start_date, end_date):
    def daterange(start_date, end_date):
        for n in range(int((end_date - start_date).days + 1)):
            yield datetime.strftime(start_date + timedelta(n), "%Y-%m-%d")

    start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date_1 = datetime.now().date() - timedelta(days=1)
    end_date_2 = datetime.strptime(end_date, "%Y-%m-%d").date()

    if end_date_1 < end_date_2:
        end_date = end_date_1
    else:
        end_date = end_date_2

    return daterange(start_date, end_date)


def scrape_days_from_api(
    start_date: str, end_date: str
) -> tuple[pd.DataFrame, pd.DataFrame, str]:
    """
    Scrapes the matrix API for a given period
    Writes outputs to raw-history bucket with folder specified by 'env'

    Parameters:
        start_date: Start date in format %Y-%m-%d
        end_date: End date in format %Y-%m-%d
            can also be 'eod' to denote end of day
    """

    url = "https://app.matrixbooking.com/api/v1/booking"
    page_size = 2500
    status = ["CONFIRMED", "TENTATIVE", "CANCELLED"]

    bookings = []

    # Authenticate session with API
    ses = requests.session()
    matrix_authenticate(ses)

    # Get booking categories available
    df_booking_categories = get_booking_categories(ses)

    # List with unique booking categories
    booking_categories = list(df_booking_categories["locationKind"])

    # Derive booking parameters
    params = make_booking_params(
        start_date,
        end_date,
        booking_categories,
        pageNum=0,
        pageSize=page_size,
        status=status,
    )

    # Scrape the first page of data
    logger.info("Scraping page 0")
    data = get_payload(ses, url, params)
    rowcount = len(data["bookings"])
    logger.info(f"Records scraped: {rowcount}")

    # Pull out the bookings and location data seperately
    bookings = data["bookings"]
    locations = data["locations"]

    i = 1
    total_rows = rowcount
    while rowcount == page_size:
        logger.info(f"Scraping page {i}")
        params = make_booking_params(
            start_date, end_date, pageNum=i, pageSize=page_size, status=status
        )
        data = get_payload(ses, url, params)
        rowcount = len(data["bookings"])
        logger.info(f"Records scraped: {rowcount}")
        if rowcount > 0:
            bookings.extend(data["bookings"])
        i += 1
        total_rows += rowcount

    logger.info(f"Retrieved {len(locations)} locations")
    logger.info(f"Retrieved {total_rows} bookings")

    raw_bookings = pd.json_normalize(bookings, sep="_").rename(
        mapper=camel_to_snake_case, axis="columns"
    )
    raw_locations = pd.json_normalize(locations, sep="_").rename(
        mapper=camel_to_snake_case, axis="columns"
    )

    return raw_bookings, raw_locations


def camel_to_snake_case(input_str: str) -> str:
    # Using regular expressions to find positions with capital letters and insert underscores
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", input_str)
    s2 = re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1)

    # Handle the case where multiple uppercase letters are present
    snake_case_str = re.sub("([a-z])([A-Z]+)", r"\1_\2", s2).lower()

    return snake_case_str


def rename_df(df: pd.DataFrame, renames: dict) -> pd.DataFrame:
    """_summary_

    Parameters
    ----------
    df :
        _description_
    renames : _type_
        _description_

    Returns
    -------
    _type_
        _description_

    Raises
    ------
    ValueError
        _description_
    """
    # Find any names that are not in the renames dict
    renames_data = [name for name in renames if name not in df.columns]
    if len(renames_data) > 0:
        logger.info(f"{renames_data} not in scraped dataframe")
    else:
        df = df.rename(columns=renames)
    return df


def fix_faulty_time_col(df, col):
    column = df[col].copy()
    # Check for missing parts (seconds, minutes, hours, microseconds)
    missing_parts = column.apply(
        lambda x: (pd.notna(x) and (len(str(x).split(":")) < 3 or "." not in str(x)))
    )

    def format_timestamp(raw_string):
        # Generate dynamic format based on missing parts
        num_parts = len(raw_string.split(":"))

        format_str = (
            "%Y-%m-%dT"
            + ":".join(["%H", "%M", "%S"][:num_parts])
            + (".%f" if "." in raw_string else "")
        )

        return pd.to_datetime(raw_string, format=format_str).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )

    column.loc[missing_parts] = column.loc[missing_parts].apply(format_timestamp)
    return column


def fix_faulty_time_cols(df):
    """_summary_

    Returns
    -------
    _type_
        _description_
    """
    bookings_metadata = Metadata.from_json("metadata/db_v2/preprod/bookings.json")
    for col in bookings_metadata:
        if "timestamp" in col["type"]:
            if col["name"] in df.columns:
                df[col["name"]] = fix_faulty_time_col(df, col["name"])
    return df


def write_raw_data_to_s3(
    bookings: pd.DataFrame, locations: pd.DataFrame, start_date: str, env: str
):
    """_summary_

    Parameters
    ----------
    bookings : _type_
        _description_
    locations : _type_
        _description_
    start_date : _type_
        _description_
    """
    raw_bookings_loc = f"{land_location}/bookings/{start_date}/raw-{start_date}.jsonl"
    raw_locations_loc = f"{land_location}/locations/{start_date}/raw-{start_date}.jsonl"
    bookings = rename_df(bookings, bookings_renames)
    bookings = fix_faulty_time_cols(bookings)
    locations = rename_df(locations, location_renames)
    writer.write(
        bookings,
        raw_bookings_loc,
    )
    writer.write(
        locations,
        raw_locations_loc,
    )
    logger.info(f"{env}: raw booking and location data written to {land_location}.")


def scrape_and_write_raw_data(start_date, env):
    bookings, locations = scrape_days_from_api(start_date, "eod")
    write_raw_data_to_s3(bookings, locations, start_date, env)

In [ ]:
pd.set_option('display.max_columns', None)

# Make API requests

## Provide booking categories (all) and status (all)

In [ ]:
ses = requests.session()
matrix_authenticate(ses)
df = get_booking_categories(ses)
booking_categories = list(df["locationKind"])
status = ["CONFIRMED", "TENTATIVE", "CANCELLED"]
params = {
    "f": "2023-12-12",
    "t": "eod",
    "bc": booking_categories,
    "status": status,
    "include": "audit",
    "pageSize": 2500,
    "pageNum": 0,
}
url = "https://app.matrixbooking.com/api/v1/booking"

# Scrape the first page of data
data = get_payload(ses, url, params)
test_df = pd.json_normalize(data.json())
print(data.url)
writer.write(
    test_df,
    "s3://alpha-dag-matrix/testing-api/called_with_status.parquet",
)

## Provide booking categories (all) but no status

In [ ]:
ses = requests.session()
matrix_authenticate(ses)
df = get_booking_categories(ses)
booking_categories = list(df["locationKind"])
status = ["CONFIRMED", "TENTATIVE", "CANCELLED"]
params = {
    "f": "2023-12-12",
    "t": "eod",
    "bc": booking_categories,
    # "status": status,
    "include": "audit",
    "pageSize": 2500,
    "pageNum": 0,
}
url = "https://app.matrixbooking.com/api/v1/booking"

# Scrape the first page of data
data = get_payload(ses, url, params)
print(data.url)
test_df = pd.json_normalize(data.json())
writer.write(
    test_df,
    "s3://alpha-dag-matrix/testing-api/called_without_status.parquet",
)

## Provide booking categories (all) but a single status 

### Confirmed

In [ ]:
ses = requests.session()
matrix_authenticate(ses)
df = get_booking_categories(ses)
booking_categories = list(df["locationKind"])
status = ["CONFIRMED"]
params = {
    "f": "2023-12-12",
    "t": "eod",
    "bc": booking_categories,
    "status": status,
    "include": "audit",
    "pageSize": 2500,
    "pageNum": 0,
}
url = "https://app.matrixbooking.com/api/v1/booking"

# Scrape the first page of data
data = get_payload(ses, url, params)
print(data.url)
test_df = pd.json_normalize(data.json())
writer.write(
    test_df,
    "s3://alpha-dag-matrix/testing-api/called_with_status_confirmed.parquet",
)

### Tentative

In [ ]:
ses = requests.session()
matrix_authenticate(ses)
df = get_booking_categories(ses)
booking_categories = list(df["locationKind"])
status = ["TENTATIVE"]
params = {
    "f": "2023-12-12",
    "t": "eod",
    "bc": booking_categories,
    "status": status,
    "include": "audit",
    "pageSize": 2500,
    "pageNum": 0,
}
url = "https://app.matrixbooking.com/api/v1/booking"

# Scrape the first page of data
data = get_payload(ses, url, params)
print(data.url)
test_df = pd.json_normalize(data.json())
writer.write(
    test_df,
    "s3://alpha-dag-matrix/testing-api/called_with_status_tentative.parquet",
)

### Cancelled

In [ ]:
ses = requests.session()
matrix_authenticate(ses)
df = get_booking_categories(ses)
booking_categories = list(df["locationKind"])
status = ["CANCELLED"]
params = {
    "f": "2023-12-12",
    "t": "eod",
    "bc": booking_categories,
    "status": status,
    "include": "audit",
    "pageSize": 2500,
    "pageNum": 0,
}
url = "https://app.matrixbooking.com/api/v1/booking"

# Scrape the first page of data
data = get_payload(ses, url, params)
print(data.url)
test_df = pd.json_normalize(data.json())
writer.write(
    test_df,
    "s3://alpha-dag-matrix/testing-api/called_with_status_cancelled.parquet",
)

## Provide no booking categories and no status

In [ ]:
ses = requests.session()
matrix_authenticate(ses)
df = get_booking_categories(ses)
booking_categories = list(df["locationKind"])
status = ["CONFIRMED", "TENTATIVE", "CANCELLED"]
params = {
    "f": "2023-12-12",
    "t": "eod",
    # "bc": booking_categories,
    # "status": status,
    "include": "audit",
    "pageSize": 2500,
    "pageNum": 0,
}
url = "https://app.matrixbooking.com/api/v1/booking"

# Scrape the first page of data
data = get_payload(ses, url, params)
test_df = pd.json_normalize(data.json())
print(data.url)
writer.write(
    test_df,
    "s3://alpha-dag-matrix/testing-api/called_with_no_bc_no_status.parquet",
)

## Read back data

In [ ]:
# Read in dataframes
without_status_df = reader.read(
    "s3://alpha-dag-matrix/testing-api/called_without_status.parquet"
)
with_status_df = reader.read(
    "s3://alpha-dag-matrix/testing-api/called_with_status.parquet"
)
with_status_confirmed_df = reader.read("s3://alpha-dag-matrix/testing-api/called_with_status_confirmed.parquet"
)
with_status_tent_df = reader.read("s3://alpha-dag-matrix/testing-api/called_with_status_tentative.parquet"
)
with_status_cancelled_df = reader.read("s3://alpha-dag-matrix/testing-api/called_with_status_cancelled.parquet"
)
without_bc_or_status_df = reader.read("s3://alpha-dag-matrix/testing-api/called_with_no_bc_no_status.parquet"
)

## Summarise data

In [ ]:
def get_info(df):
    unique_locations = len(df.locationId.unique())
    unique_organisations = len(df["organisation.id"].unique())
    unique_booking_ids = len(df.id.unique())
    status_returns = df.status.unique()
    location_kind_returns = df.locationKind.unique()
    sources = df.source.unique()
    print(
        f"""
            No of Unique Locations: {unique_locations}
            No of Unique organisations: {unique_organisations}
            No of Unique bookings ids: {unique_booking_ids}
            Different Statuses: {status_returns}
            Different Locations: {location_kind_returns}           
          """
    )

All booking types provided explicitly

In [ ]:
# No status provided 
get_info(without_status_df)

In [ ]:
# All status provided
get_info(with_status_df)

In [ ]:
# Status confirmed only
get_info(with_status_confirmed_df)

In [ ]:
# Status tentative only
get_info(with_status_tent_df)

In [ ]:
# Status cancelled 
get_info(with_status_cancelled_df)

In [ ]:
get_info(without_bc_or_status_df)

In [ ]:
# Volumes of status compared to booking type
pd.crosstab(without_status_df['locationKind'], without_status_df['status'], dropna=False, margins=True)


## Combine the data

### No status vs all status provided

In [ ]:
# Merge the data and retain an indicator 
df_full = pd.merge(without_status_df,
                   with_status_df,
                   how='outer',
                   left_on='id',
                   right_on='id',
                   indicator=True
                    )
# Volumes
df_full['_merge'].value_counts()

### With status provided

We expect those in 'right only' to be cancelled only

In [ ]:
# For those in 'with status only' we expect all to be cancelled
df_right = df_full[df_full["_merge"]=='right_only'].copy()
df_right['status_y'].value_counts()

### With status not provided (Only)

What can we say about those that weren't common to both?

In [ ]:
# For those 'left only' what organisations are they?
df_left = df_full[df_full["_merge"]=='left_only'].copy()
df_left['organisation.name_x'].value_counts()

# This includes some 43 - need to work out why these 43s are not in the other....

In [ ]:
df_left_43 = df_left[df_left['organisation.id_x'] == 43]
df_left_43.shape

In [ ]:
df_left_43.head()

## Status - confirmed

In [ ]:
with_status_confirmed_df['organisation.id'].value_counts()

In [ ]:
# Are all 'confirmed' common to the 'without status' query?
df_confirmed_common = pd.merge(with_status_confirmed_df,
                                without_status_df,
                                how='outer',
                                left_on = 'id',
                                right_on = 'id',
                                indicator=True)
df_confirmed_common['_merge'].value_counts()

## Status - Tentative

In [ ]:
with_status_tent_df['organisation.id'].value_counts()

In [ ]:
# Are all 'confirmed' common to the 'without status' query?
df_tent_common = pd.merge(with_status_tent_df,
                                without_status_df,
                                how='outer',
                                left_on = 'id',
                                right_on = 'id',
                                indicator=True)
df_tent_common['_merge'].value_counts()

## Status - Cancelled

In [ ]:
with_status_cancelled_df.shape

In [ ]:
# Checking whether we get any more 'cancelled' bookings if we provide 
# the status explicitly
df_cancelled_common = pd.merge(with_status_cancelled_df,
                               with_status_df,
                               how='inner',
                               left_on='id',
                               right_on='id',
                               indicator=True

)
df_cancelled_common["_merge"].value_counts()

## Compare results where we don't provide booking categories or status

In [ ]:
# Expect everything to match
df_nobc_no_status_common = pd.merge(without_bc_or_status_df,
                                    without_status_df,
                                    how='outer',
                                    left_on='id',
                                    right_on='id',
                                    indicator=True)
df_nobc_no_status_common['_merge'].value_counts()